In [11]:
import nltk
import textstat
import re
import os
import string

import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
data = pd.read_excel("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\Output Data Structure.xlsx")
url = data.URL.to_list()
url_id = data.URL_ID.to_list()
url_id = [int(x) if x%int(x) == 0 else x for x in url_id]

# Test

In [3]:
def extract_words(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Find words using regex
    words = re.findall(r'\b\w+\b', content)
    return words


def extract_capital_words(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # This pattern matches any word that is ALL CAPS.
    pattern_all_caps = r'\b[A-Z]+\b'
    all_caps_words = re.findall(pattern_all_caps, content)
    return all_caps_words

In [5]:
neg_words = set(extract_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\Blackcoffer Internship New\\Important words\\negative-words.txt"))
pos_words = set(extract_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\Blackcoffer Internship New\\Important words\\positive-words.txt"))

stop_words1 = extract_capital_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\Important words\\StopWords_Auditor.txt")
stop_words2 = extract_capital_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\Important words\\StopWords_Currencies.txt")
stop_words3 = extract_capital_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\Important words\\StopWords_DatesandNumbers.txt")
stop_words4 = extract_capital_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\Important words\\StopWords_Generic.txt")
stop_words5 = extract_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\Important words\\StopWords_GenericLong.txt")
stop_words6 = extract_capital_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\Important words\\StopWords_Geographic.txt")
stop_words7 = extract_capital_words("C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\Important words\\StopWords_Names.txt")

stopwords = set(stop_words1 + stop_words2 + stop_words3 + stop_words4 + stop_words5 + stop_words6 + stop_words7)

# NLP

In [8]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict, stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('punkt')
nltk.download('cmudict')
nltk.download('vader_lexicon')
nltk.download('stopwords')

def perform_nlp_analysis(file_path):
    try:
        # Read the content of the file
        with open(file_path, 'r') as file:
            text = file.read()

        # Sentimental Analysis
        cleaned_text = clean_text(text)
        derived_variables = extract_derived_variables(cleaned_text)

        # Print the derived variables
        print("Derived Variables:")
        for variable, value in derived_variables.items():
            print(f'{variable}: {value}')

        # NLP Metrics Analysis
        nlp_metrics = calculate_nlp_metrics(text)

        new_row = pd.DataFrame(nlp_metrics)
        
        return new_row

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

def clean_text(text):

    words = word_tokenize(text)
    cleaned_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stopwords]
    return ' '.join(cleaned_words)

def create_positive_negative_dict():
    # Assuming you have lists of positive and negative words
    positive_words = pos_words  # Add more positive words
    negative_words = neg_words  # Add more negative words
    return {'positive': positive_words, 'negative': negative_words}

def extract_derived_variables(text):
    positive_negative_dict = create_positive_negative_dict()
    positive_words = positive_negative_dict['positive']
    negative_words = positive_negative_dict['negative']

    positive_word_count = sum(1 for word in text.split() if word in positive_words)
    negative_word_count = sum(1 for word in text.split() if word in negative_words)

    ratio_positive_negative = positive_word_count / (negative_word_count + 1)  # Adding 1 to avoid division by zero

    return {
        'POSITIVE WORD COUNT': positive_word_count,
        'NEGATIVE WORD COUNT': negative_word_count,
        'RATIO POSITIVE/NEGATIVE WORDS': ratio_positive_negative
    }

def calculate_nlp_metrics(text):
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(text)

    avg_sentence_length = len(word_tokenize(text)) / len(sent_tokenize(text))
    complex_word_count = sum(1 for word in text.split() if syllable_count(word) > 2)
    percentage_complex_words = (complex_word_count / len(text.split())) * 100

    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    avg_words_per_sentence = len(text.split()) / len(sent_tokenize(text))
    avg_word_length = sum(len(word) for word in text.split()) / len(text.split())
    syllable_per_word = sum(syllable_count(word) for word in text.split()) / len(text.split())

    personal_pronouns = sum(1 for word in text.split() if word.lower() in ['i', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves'])

    return {
        'POSITIVE SCORE': sentiment_scores['pos'],
        'NEGATIVE SCORE': sentiment_scores['neg'],
        'POLARITY SCORE': sentiment_scores['compound'],
        'SUBJECTIVITY SCORE': 1 - abs(sentiment_scores['compound']),
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
        'FOG INDEX': fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
        'COMPLEX WORD COUNT': complex_word_count,
        'WORD COUNT': len(text.split()),
        'SYLLABLE PER WORD': syllable_per_word,
        'PERSONAL PRONOUNS': personal_pronouns,
        'AVG WORD LENGTH': avg_word_length
    }

def syllable_count(word):
    d = cmudict.dict()
    return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]]) if word.lower() in d else 0




[nltk_data] Downloading package punkt to C:\Users\Asfaan
[nltk_data]     Hussain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to C:\Users\Asfaan
[nltk_data]     Hussain\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Asfaan
[nltk_data]     Hussain\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Asfaan
[nltk_data]     Hussain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
exclude = string.punctuation

def write_text_without_blackcoffer(file_path):
    try:
        # Read the content of the file
        with open(file_path, 'r') as file:
            content = file.read()

        # Find the starting index of "Blackcoffer"
        index_blackcoffer = content.rfind('Blackcoffer')

        # Remove text starting from "Blackcoffer" until the end
        modified_content = content[:index_blackcoffer]

        # Write the modified content back to the file
        with open(file_path, 'w') as file:
            file.write(modified_content)

        return modified_content

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")



def remove_punc(text):
    return text.translate(str.maketrans("","",exclude))

In [13]:
df = pd.DataFrame()
for i in url_id:
    directory = "C:\\Users\\Asfaan Hussain\\Desktop\\INTERNSHALA\\Blackcoffer Internship New\\TextFiles"
    filename = '{}.txt'.format(i)
    filepath = os.path.join(directory, filename)
    new_row = perform_nlp_analysis(filepath)
    print(new_row)

An error occurred: argument of type 'LazyCorpusLoader' is not iterable
None
An error occurred: argument of type 'LazyCorpusLoader' is not iterable
None
An error occurred: argument of type 'LazyCorpusLoader' is not iterable
None
An error occurred: 'charmap' codec can't decode byte 0x9d in position 135: character maps to <undefined>
None
An error occurred: 'charmap' codec can't decode byte 0x9d in position 135: character maps to <undefined>
None
An error occurred: 'charmap' codec can't decode byte 0x9d in position 652: character maps to <undefined>
None
An error occurred: 'charmap' codec can't decode byte 0x9d in position 100: character maps to <undefined>
None
An error occurred: 'charmap' codec can't decode byte 0x9d in position 992: character maps to <undefined>
None
An error occurred: 'charmap' codec can't decode byte 0x9d in position 315: character maps to <undefined>
None
An error occurred: 'charmap' codec can't decode byte 0x9d in position 129: character maps to <undefined>
None
An